In [1]:
!git clone https://github.com/nkt780426/3d_face_recognition_magface.git

Cloning into '3d_face_recognition_magface'...
remote: Enumerating objects: 611, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 611 (delta 46), reused 63 (delta 21), pack-reused 516 (from 1)
Receiving objects: 100% (611/611), 98.30 MiB | 43.20 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [2]:
!mv 3d_face_recognition_magface/going_modular .

In [3]:
!gdown --quiet --folder https://drive.google.com/drive/folders/1-1KWxxoID1SrKzs1BYvEj6whqBzz1LtW?usp=sharing

In [4]:
import warnings
warnings.filterwarnings("ignore")
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A
import os
import torch

from going_modular.dataloader.multitask import create_concatv2_multitask_datafetcher
from going_modular.model.MTLFaceRecognition import MTLFaceRecognition
from going_modular.model.ConcatMTLFaceRecognition import ConcatMTLFaceRecognitionV2
from going_modular.loss.ConcatMultiTaskLoss import ConcatMultiTaskLoss
from going_modular.train_eval.concat_train import fit
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    'type': 'concat2',
    
    # Thư mục
    'dataset_dir': '/kaggle/input/hoangvn-3dmultitask',
    'dataset_dir_1': '/kaggle/input/hoangvn-3dmultitask/Albedo',
    'dataset_dir_2': '/kaggle/input/hoangvn-3dmultitask/Depth_Map',
    'checkpoint_dir': '/kaggle/working/checkpoint/multi/',
    'checkpoint_1': '/kaggle/working/multi/albedo/checkpoint.pth',
    'checkpoint_2': '/kaggle/working/multi/depthmap/checkpoint.pth',
        
    # Cấu hình train
    'device': device,
    'epochs': 119,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 256,
    'base_lr': 1e-4,
    
    # Cấu hình network
    'backbone': 'miresnet18',
    'embedding_size': 512,
    'num_classes': None,
    'loss_gender_weight': 30,
    'loss_emotion_weight': 5,
    'loss_pose_weight': 30,
    'loss_spectacles_weight': 10,
    'loss_facial_hair_weight': 10,
}

CONFIGURATION['num_classes'] = len(os.listdir('/kaggle/input/hoangvn-3dmultitask/Albedo/train'))

train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
], additional_targets={
    'image2': 'image'
})


test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
], additional_targets={
    'image2': 'image'
})

train_dataloader, test_dataloader = create_concatv2_multitask_datafetcher(CONFIGURATION, train_transform, test_transform)

checkpoint_1 = torch.load(CONFIGURATION['checkpoint_1'], map_location=torch.device('cpu'))
mtl_backbone1 = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])
mtl_backbone1.load_state_dict(checkpoint_1['model_state_dict'])

checkpoint_2 = torch.load(CONFIGURATION['checkpoint_2'], map_location=torch.device('cpu'))
mtl_backbone2 = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])
mtl_backbone2.load_state_dict(checkpoint_2['model_state_dict'])

model = ConcatMTLFaceRecognitionV2(mtl_backbone1, mtl_backbone2, CONFIGURATION['num_classes'])

for param in model.mtl_backbone1.parameters():
    param.requires_grad = False
    
for param in model.mtl_backbone2.parameters():
    param.requires_grad = False

criterion = ConcatMultiTaskLoss(os.path.join(CONFIGURATION['dataset_dir'], 'train_set.csv'), CONFIGURATION)

optimizer = Adam(model.parameters(), lr=CONFIGURATION['base_lr'])
# Khởi tạo scheduler
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=40, T_mult=1, eta_min=1e-6)

earlystop_dir = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models')

early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc', 'euclidean_auc'],
    patience=1000,
    mode='max',
    verbose=0,
    save_dir=earlystop_dir,
    start_from_epoch=0
)      
checkpoint_path = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)


fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    train_dataloader=train_dataloader, 
    test_dataloader=test_dataloader, 
    criterion=criterion,
    optimizer=optimizer,    
    scheduler=scheduler, 
    early_stopping=early_stopping,
    model_checkpoint=modle_checkpoint
)

Epoch 1:
╒══════════════════╤════════════╤══════════════════════╕
│ Metric           │      Train │ Test                 │
╞══════════════════╪════════════╪══════════════════════╡
│ loss             │ 26.6312    │ -                    │
├──────────────────┼────────────┼──────────────────────┤
│ loss_id          │ 23.0398    │ -                    │
├──────────────────┼────────────┼──────────────────────┤
│ loss_gender      │  0.0313788 │ 0.02751845351109902  │
├──────────────────┼────────────┼──────────────────────┤
│ loss_emotion     │  0.0961826 │ 0.31612460343167187  │
├──────────────────┼────────────┼──────────────────────┤
│ loss_pose        │  0.0131723 │ 0.011946830355251828 │
├──────────────────┼────────────┼──────────────────────┤
│ loss_facial_hair │  0.102401  │ 0.11384401886413495  │
├──────────────────┼────────────┼──────────────────────┤
│ loss_spectacles  │  0.0749915 │ 0.07713575003047785  │
├──────────────────┼────────────┼──────────────────────┤
│ auc_gender       │  

In [5]:
!zip -r output.zip checkpoint/multi/

  adding: checkpoint/multi/ (stored 0%)
  adding: checkpoint/multi/concat2/ (stored 0%)
  adding: checkpoint/multi/concat2/models/ (stored 0%)
  adding: checkpoint/multi/concat2/models/checkpoint.pth (deflated 7%)
  adding: checkpoint/multi/concat2/models/best_euclidean_auc_12.pth (deflated 7%)
  adding: checkpoint/multi/concat2/models/best_cosine_auc_107.pth (deflated 7%)
  adding: checkpoint/multi/concat2/logs/ (stored 0%)
  adding: checkpoint/multi/concat2/logs/AUC_cosine_test/ (stored 0%)
  adding: checkpoint/multi/concat2/logs/AUC_cosine_test/events.out.tfevents.1736982326.723006cca3cc.18.12 (deflated 63%)
  adding: checkpoint/multi/concat2/logs/AUC_emotion_train/ (stored 0%)
  adding: checkpoint/multi/concat2/logs/AUC_emotion_train/events.out.tfevents.1736982326.723006cca3cc.18.23 (deflated 70%)
  adding: checkpoint/multi/concat2/logs/AUC_gender_test/ (stored 0%)
  adding: checkpoint/multi/concat2/logs/AUC_gender_test/events.out.tfevents.1736982326.723006cca3cc.18.16 (deflated 70